LCEL (대화내용 기억하기): 메모리 추가

임의의 체인에 메모리를 추가하는 방법을 보여줍니다. 현재 메모리 클래스를 사용할 수 있지만 수동으로 연결해야 합니다

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.chat_models import ChatOllama

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:1b")

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)


/var/folders/5d/4yvbq7s16359fdr947szsx5h0000gn/T/ipykernel_5879/3650269879.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:1b")


대화내용을 저장할 메모리인 ConversationBufferMemory 생성하고 return_messages 매개변수를 True로 설정하여, 생성된 인스턴스가 메시지를 반환하도록 합니다.

In [3]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/var/folders/5d/4yvbq7s16359fdr947szsx5h0000gn/T/ipykernel_5879/2260825137.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [4]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.


{'chat_history': []}

In [5]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

In [6]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [7]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [11]:
chain = runnable | prompt | llm

In [13]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 현우입니다."})
print(response.content)  # 생성된 응답을 출력합니다.

안녕하세요, 현우님! 만나서 반갑습니다. 저는 여러분을 돕기 위해 여기 있는 AI 챗봇입니다. 무엇을 도와드릴까요? 😊


In [14]:
memory.load_memory_variables({})

{'chat_history': []}

memory.save_context 함수는 입력 데이터(inputs)와 응답 내용(response.content)을 메모리에 저장하는 역할을 합니다. 이는 AI 모델의 학습 과정에서 현재 상태를 기록하거나, 사용자의 요청과 시스템의 응답을 추적하는 데 사용될 수 있습니다.

In [18]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 현우입니다."}, {"ai": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})


{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 현우입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 현우님! 만나서 반갑습니다. 저는 여러분을 돕기 위해 여기 있는 AI 챗봇입니다. 무엇을 도와드릴까요? 😊', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='만나서 반갑습니다. 제 이름은 현우입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송합니다. 저는 기억할 수 없습니다. 저는 당신과 대화하는 데만 집중하고 있습니다. 😊', additional_kwargs={}, response_metadata={})]}

In [19]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)

아, 기억났어요! 당신의 이름은 현우였죠. 😊


커스텀 ConversationChain 구현 예시

In [20]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_core.output_parsers import StrOutputParser

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:1b")

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [21]:
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer


In [22]:

conversation_chain = MyConversationChain(llm, prompt, memory)


In [23]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 현우 입니다.")

"안녕하세요, 현우님! 만나서 정말 반갑습니다. 저는 여러분을 돕기 위해 여기 있는 AI 챗봇입니다. 무엇을 도와드릴까요? 😊 \n\n(Hello, Hyunwoo! It's really nice to meet you. I am an AI chatbot here to help you. What can I do for you?)"

In [24]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'제 이름은 챗봇입니다. 😊'

In [25]:
conversation_chain.invoke("앞으로는 영어로만 답변해주세요 알겠어요?")

'네, 알겠습니다. 앞으로는 영어로만 답변하겠습니다. \n\n(Yes, I understand. From now on, I will only answer in English.)'

In [26]:
conversation_chain.invoke("제 이름을 다시 한 번 말해주세요")

'안녕하세요, 현우님. 😊'

In [27]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세요? 만나서 반갑습니다. 제 이름은 현우 입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="안녕하세요, 현우님! 만나서 정말 반갑습니다. 저는 여러분을 돕기 위해 여기 있는 AI 챗봇입니다. 무엇을 도와드릴까요? 😊 \n\n(Hello, Hyunwoo! It's really nice to meet you. I am an AI chatbot here to help you. What can I do for you?)", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐라고요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='제 이름은 챗봇입니다. 😊', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='앞으로는 영어로만 답변해주세요 알겠어요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 알겠습니다. 앞으로는 영어로만 답변하겠습니다. \n\n(Yes, I understand. From now on, I will only answer in English.)', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름을 다시 한 번 말해주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 현우님. 😊', additional_kwargs={}, response_metadata={})]